In [1]:
%matplotlib inline
import cv2
from pylab import *
import torch
from torch import nn
from models import LinkNet34
import torchvision.transforms as transforms
from torch.autograd import Variable
import sys
from PIL import Image, ImageFilter


import cv2

In [2]:
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
model = LinkNet34()
# model.load_state_dict(torch.load('linknet.pth'))
model.load_state_dict(torch.load('linknet.pth', map_location=lambda storage, loc: storage))

model.eval()
model.to(device)
# summary(model, (3, 256, 256))
1

1

In [3]:
img_transform = transforms.Compose([
    transforms.Resize((256,256)),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225], inplace=False)
])
t = transforms.Resize(size=(256,256))


In [4]:
def smooth_mask(mask):
    im = Image.fromarray(mask)
    im2 = im.filter(ImageFilter.MinFilter(3))
    im3 = im2.filter(ImageFilter.MaxFilter(5))
    return np.array(im3)

In [5]:

def plt_mask(path):
    _img = Image.open(path)
    a = img_transform(_img)
    a = a.unsqueeze(0)
    imgs = Variable(a.to(device))
    pred = model(imgs)
    img = np.array(_img)
    mask = pred > 0.5
    mask = mask.squeeze()
    mask = mask.cpu().numpy()
    img=np.array(t(Image.fromarray(img)))
    img[mask==0]=170
    plt.imshow(img)

In [ ]:
p = '/media/nasir/Drive1/datasets/test/0.jpeg'
model.eval()
time_1 = time.time()

plt_mask(p)
time_2 = time.time()
print(f'time: {time_2 - time_1}')

In [ ]:
p = '/media/nasir/Drive1/datasets/test/0.jpeg'
model.eval()
time_1 = time.time()

plt_mask(p)
time_2 = time.time()
print(f'time: {time_2 - time_1}')

In [6]:
camera = cv2.VideoCapture(0)

model.to(device)
time.sleep(1)
model.eval()
(grabbed, frame) = camera.read()

time_1 = time.time()
frames_count = 0
while grabbed:
    (grabbed, orig) = camera.read()
    shape = orig.shape[0:2]
    frame = cv2.cvtColor(orig, cv2.COLOR_BGR2RGB)
    frame = cv2.resize(frame,(256,256), cv2.INTER_LINEAR )
    if not grabbed:
        continue
    
    k = cv2.waitKey(1)
    if k != -1:    # any key to stop
        break

    a = img_transform(Image.fromarray(frame))
    a = a.unsqueeze(0)

    imgs = Variable(a.to(dtype=torch.float, device=device))
    pred = model(imgs)
    
    pred= torch.nn.functional.interpolate(pred, size=[shape[0], shape[1]])
    mask = pred.data.cpu().numpy()
    mask = mask.squeeze()    
    mask = mask > 0.5
    orig[mask==0]=170
    cv2.imshow('frame', orig)
    if frames_count % 30 == 0:
        time_2 = time.time()
        print(f'FPS: {frames_count/(time_2-time_1)}')
    frames_count+=1
    
cv2.destroyAllWindows()
camera.release()


FPS: 0.0
FPS: 2.737222040205968
FPS: 2.794216723700575
FPS: 2.8058874052816836


KeyboardInterrupt: 